# Agent Based Model: Main Notebook

In [1172]:
import numpy.random as rnd
import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt
import random

#### Workflow Notes 
* Draw Students with values within the four dimensions 
* Function that creates a study group (draw 4 students) -> Measure homogeniety 
* Task 


## Importing The Study Groups that are to be Tested

In [1173]:
homogenous = pd.read_csv("Conditions/Homogenous.csv")
fiftyfifty = pd.read_csv("Conditions/FiftyFifty.csv")
heterogenous = pd.read_csv("Conditions/Heterogenous.csv")


for i in (range(len(homogenous.index))):
    print(list(homogenous.iloc[i][1:5]))


['ESFJ', 'ESFJ', 'ESFJ', 'ESFJ']
['ESTJ', 'ESTJ', 'ESTJ', 'ESTJ']
['ISFJ', 'ISFJ', 'ISFJ', 'ISFJ']
['ISTJ', 'ISTJ', 'ISTJ', 'ISTJ']
['ENFP', 'ENFP', 'ENFP', 'ENFP']
['ENFJ', 'ENFJ', 'ENFJ', 'ENFJ']
['INFP', 'INFP', 'INFP', 'INFP']
['INFJ', 'INFJ', 'INFJ', 'INFJ']
['ENTP', 'ENTP', 'ENTP', 'ENTP']
['ENTJ', 'ENTJ', 'ENTJ', 'ENTJ']
['INTP', 'INTP', 'INTP', 'INTP']
['INTJ', 'INTJ', 'INTJ', 'INTJ']
['ESFP', 'ESFP', 'ESFP', 'ESFP']
['ESTP', 'ESTP', 'ESTP', 'ESTP']
['ISFP', 'ISFP', 'ISFP', 'ISFP']
['ISTP', 'ISTP', 'ISTP', 'ISTP']


## Creating Study Groups

#### Practical Functions

In [1174]:
def n_sampler(size):
    '''
    sample a number of random numbers from the beta distribution
    '''
    max_vals = []
    min_vals = []

    beta_dist = rnd.beta(1.5, 1.5, size)

    for i in range(size):
        if beta_dist[i] >= 0.5:
            max_vals.append(beta_dist[i])
        else:
            min_vals.append(beta_dist[i])
    
    return max_vals, min_vals

max_vals, min_vals = n_sampler(1000000)

In [1175]:
def data_collect(studygroup, student_list):
    '''
    function to collect data from the simulation
    '''
    Name_list = []
    extraversion_list = []
    sensing_list = []
    thinking_list = []
    judging_list = []
    academic_list = []
    
    for student in studygroup:
        Name_list.append(student.Name)
        extraversion_list.append(student.ExScore)
        sensing_list.append(student.SeScore) 
        thinking_list.append(student.ThScore)
        judging_list.append(student.JuScore)
        academic_list.append(student.Academic_Skill)

    data = pd.DataFrame({'Name': Name_list, 
                        'type': student_list, 
                        'E/I': extraversion_list, 
                        'S/N': sensing_list,
                        'T/F': thinking_list,
                        'J/P': judging_list, 
                        'Academic': academic_list})
    
    return data
    

#### Create Study Group

In [1176]:
## Defining the Students ##
class Student():
    def __init__(self, Name, Ex, Se, Th, Ju):
        self.Name  = Name

        ## Personality Traits ##
        self.Ex = Ex #Extraversion vs Introversion dimension
        self.Se = Se #Sensing vs Intuition dimension
        self.Th = Th #Thinking vs Feeling dimension
        self.Ju = Ju #Judging vs Perceiving dimension
        self.Type = Ex + Se + Th + Ju

        ## Personality Scores calculated with the personality() function##
        self.ExScore = 0
        self.SeScore = 0
        self.ThScore = 0
        self.JuScore = 0
        
        self.Scores = [] #list of all personality scores

        ## Academic Skills ##
        self.Academic_Skill = 0

        ## Own Solution ##
        self.Ind_Solution = []

def personality(student):
    # Extraversion vs. Introversion
    if student.Ex == "E":
        student.ExScore = max_vals[0]
        del max_vals[0]
    else:
        student.ExScore = min_vals[0]
        del min_vals[0]
    
    # Sensing vs. Intuition
    if student.Se == "S":
        student.SeScore = max_vals[0]
        del max_vals[0]
    else:
        student.SeScore = min_vals[0]
        del min_vals[0]
    
    # Thinking vs. Feeling
    if student.Th == "T":
        student.ThScore = max_vals[0]
        del max_vals[0]
    else:
        student.ThScore = min_vals[0]
        del min_vals[0]

    # Judging vs. Perceiving
    if student.Ju == "J":
        student.JuScore = max_vals[0]
        del max_vals[0]
    else:
        student.JuScore = min_vals[0]
        del min_vals[0]
    
    student.Scores = [student.ExScore, student.SeScore, student.ThScore, student.JuScore]

def skills(student):
    student.Academic_Skill = (1-student.SeScore)*0.33 + student.JuScore*0.33 + (rnd.beta(8, 2, 1)[0]*0.33)
    student.Compromising = (1-student.ThScore)*0.33 + student.ExScore*0.33 + (rnd.beta(8, 2, 1)[0]*0.33)

In [1177]:
def StudyGroup(student_list):
    '''
    Create a study group of students
    '''
    studygroup = []
    names = ["Alfa", "Bravo", "Charlie", "Delta"]

    for i in range(len(student_list)):
        student = Student(names[i], student_list[i][0], student_list[i][1], student_list[i][2], student_list[i][3])
        personality(student)
        skills(student)
        studygroup.append(student)
    
    return data_collect(studygroup, student_list), studygroup

## Creating the ABM

### Generating True Solution

In [1178]:
def true_solution_generator(n_part_exercises, range_elements):
    '''
    create a list of random numbers that will serve as the true solution that the agents need to find 
    '''
    true_solution = []
    for i in range(n_part_exercises):
        true_solution.append(random.randint(range_elements[0], range_elements[1]))

    return true_solution, range_elements

### Generating Individual Solutions

In [1179]:
def individual_solutions_generator(studygroup, true_solution, range_elements):
    '''
    #function to calculate the individual solutions of the agents given a study group dataframe and a true solution
    '''
    
    all_solutions = []

    for student in studygroup[1]: # loop for each individual
        Ind_Solution_lst = []
        Ind_Solution_attr_lst = []
        
        for i in range(len(true_solution)): # loop for each part-exercise
            coin_toss = np.random.binomial(1, (student.Academic_Skill*0.5) + 0.5, 1)[0] # biased-coin flip
            if coin_toss == 1:
                Ind_Solution_lst.append(true_solution[i])
                Ind_Solution_attr_lst.append(true_solution[i])
            else:
                Ind_Solution_lst.append(random.randint(range_elements[0], range_elements[1]))
                Ind_Solution_attr_lst.append(random.randint(range_elements[0], range_elements[1]))
        
        all_solutions.append(Ind_Solution_lst)
        student.Ind_Solution = Ind_Solution_attr_lst
    
    return all_solutions

In [1180]:
def agent_type_equality(agent_a, agent_b):
    equality_score = 0 # number from 0 to 1
    for i in range(4):
        if agent_a[i] == agent_b[i]:
            equality_score += 0.25
    return equality_score 

In [1181]:
A = "ESTJ"
B = "ISTJ"
agent_type_equality(A, B)

0.75

## Collaborative Problem Solving

##### Workflow notes
1. Who presents their solution e.g. agent A presents their solution to agent B, C, D -> THE PROPOSED SOLUTION
    -> Based on Extraversion score (Highest extraversion score is the most likely to present their solution)
2. According to an Agreeableness score (Social score for now) of the other agents (and maybe a Trustworthiness score of agent proposing), agents will update their solution
3. The solutions of the agents will be checked, if all they agree, this is their final solution. If not, the process will be repeated for a max of X ticks. If the groups do not converge, an accuracy score will still be calculated. 

In [1182]:
def collaborative_solution(studygroup, max_ticks):
    max_ticks = max_ticks #turns in the simulation
    n_ticks = 0
    consensus = False
    
    # extracting all 4 students
    Alfa = studygroup[1][0]
    Bravo = studygroup[1][1]
    Charlie = studygroup[1][2]
    Delta = studygroup[1][3]

    student_list = [Alfa, Bravo, Charlie, Delta]
    individual_solutions = [Alfa.Ind_Solution, Bravo.Ind_Solution, Charlie.Ind_Solution, Delta.Ind_Solution]

    while (n_ticks != max_ticks):
        # Starting a Round
        presenter_name = random.choices([Alfa.Name, Bravo.Name, Charlie.Name, Delta.Name], weights = [Alfa.ExScore, Bravo.ExScore, Charlie.ExScore, Delta.ExScore], k = 1)[0] # selecting the presenter of the round based on weighted random draw from extraversion scores
        
        Proposed_Solution = eval(presenter_name).Ind_Solution
        #print(n_ticks, presenter_name, Proposed_Solution)

        for student in student_list:
            if student == eval(presenter_name):
                continue
            
            similarity_score = agent_type_equality(eval(presenter_name).Type, student.Type) # evaluate how equal the two agents are in personality. This bases the coinflip
            similarity_coin_toss = np.random.binomial(1, similarity_score, 1)[0]
            
            if similarity_coin_toss == 0:
                coin_toss = np.random.binomial(1, 0.5, 1)[0] # giving a 50% chance of not ending loop if you lose on similarity score.
                if coin_toss == 1:
                    continue
                
            for i in range(len(Proposed_Solution)): # looping through all part-exercises and evaluating against proposed solution.
                coin_toss = np.random.binomial(1, (student.Compromising*0.25 + eval(presenter_name).Academic_Skill*0.25), 1)[0] #high social skill has a greater chance of accepting the proposal and if the proposer has higher academic skill.
                if coin_toss == 1:
                    student.Ind_Solution[i] = Proposed_Solution[i]
                else:
                    student.Ind_Solution[i] = student.Ind_Solution[i]

        n_ticks += 1 #adding a tick to the simulation

        if all(x==individual_solutions[0] for x in individual_solutions): # initializes a break of while-loop if consensus has been reached (i.e. all solutions are the same)
            consensus = True
            break
    
    if consensus == True:
        final_group_solution = individual_solutions[0] # take one of the solutions from the group if they are all the same
    
    if consensus == False:
        final_group_solution = Proposed_Solution



    #print("Number of iterations: " + str(n_ticks))
    return final_group_solution, n_ticks, consensus # returns their collective solution


In [1183]:
def solution_evaluator(proposed_solution, true_solution):
    correct_part_exercise = 0
    wrong_part_exercise = 0

    for i in range(len(true_solution)):
        if proposed_solution[i] == true_solution[i]:
            correct_part_exercise += 1
        else:
            wrong_part_exercise += 1

    return correct_part_exercise / (correct_part_exercise + wrong_part_exercise)


In [1184]:
def adaptation_degree(individual_solution, group_solution):
    '''
    #function to calculate the adaptation degree of the agents
    '''
    adapted = 0
    not_adapted = 0

    for i in range(len(individual_solution)):
        if individual_solution[i] == group_solution[i]:
            not_adapted += 1
        else:
            adapted += 1
    
    adaptation = adapted / (adapted + not_adapted)

    return adaptation

In [1185]:
def synergy_calculator(individual_accuracies, group_accuracy):
    '''
    #function to calculate the gain of the agents
    '''
    avg_of_indi_accuracy = np.mean(individual_accuracies)
    max_of_indi_accuracy = np.max(individual_accuracies)
    
    weak_synergy = group_accuracy - avg_of_indi_accuracy
    strong_synergy = group_accuracy - max_of_indi_accuracy

    return weak_synergy, strong_synergy

In [1186]:
def diversity_score_calculator(student_list):
    diversity_score = 0

    for i in range(4):
        if (student_list[0][i] == student_list[1][i] == student_list[2][i] == student_list[3][i]):
            diversity_score += 0
        elif (student_list[0][i] == student_list[1][i] == student_list[2][i]) or (student_list[0][i] == student_list[1][i] == student_list[3][i]) or (student_list[1][i] == student_list[2][i] == student_list[3][i]) or (student_list[0][i] == student_list[2][i] == student_list[3][i]):
            diversity_score += 1
        else:
            diversity_score += 2
    
    return diversity_score

# TEST RUN

In [1187]:
# Creating the Study Group
studygroup1 = StudyGroup(student_list_test)

In [1188]:
diversity_score_calculator(student_list_test)

7

In [1189]:
# Seeing their Generated Personality Scores
print(studygroup1[0])

      Name  type       E/I       S/N       T/F       J/P  Academic
0     Alfa  ESFP  0.809663  0.550679  0.351181  0.307924  0.577451
1    Bravo  ESFJ  0.695214  0.726146  0.240172  0.776741  0.645478
2  Charlie  INTP  0.192562  0.411793  0.782302  0.243381  0.558861
3    Delta  INTP  0.079059  0.315904  0.938174  0.010562  0.490654


In [1202]:
# Generating a True Solution
true_solution_test = true_solution_generator(10, [1, 100])

In [1203]:
# Displaying the True Solution
true_solution_test[0]

[76, 37, 40, 45, 90, 87, 76, 90, 27, 31]

In [1204]:
# Generating Individual Solutions
all_solutions = individual_solutions_generator(studygroup1, true_solution_test[0], true_solution_test[1])
all_solutions

[[76, 15, 40, 45, 77, 87, 76, 90, 27, 31],
 [76, 41, 16, 45, 83, 87, 76, 42, 27, 31],
 [76, 37, 40, 45, 90, 67, 76, 51, 27, 31],
 [76, 84, 7, 11, 90, 87, 47, 90, 27, 31]]

In [1205]:
# Printing Individual Solutions
for student in studygroup1[1]:
    print(student.Name, student.Ind_Solution)

Alfa [76, 99, 48, 45, 27, 87, 76, 90, 27, 31]
Bravo [76, 6, 10, 45, 62, 87, 76, 48, 27, 31]
Charlie [76, 37, 40, 45, 90, 71, 76, 56, 27, 31]
Delta [76, 28, 62, 14, 90, 87, 64, 90, 27, 31]


In [1206]:
# Printing Indivudal Accuracies
for student in studygroup1[1]:
    print(student.Name, solution_evaluator(student.Ind_Solution, true_solution_test[0]))

Alfa 0.7
Bravo 0.6
Charlie 0.8
Delta 0.6


In [1207]:
# Creating Group Solution
group_solution = collaborative_solution(studygroup1, 100)

In [1208]:
print(group_solution)
print(all_solutions)

([76, 6, 10, 45, 62, 87, 76, 48, 27, 31], 19, True)
[[76, 15, 40, 45, 77, 87, 76, 90, 27, 31], [76, 41, 16, 45, 83, 87, 76, 42, 27, 31], [76, 37, 40, 45, 90, 67, 76, 51, 27, 31], [76, 84, 7, 11, 90, 87, 47, 90, 27, 31]]


In [1209]:
# Printing Group Accuracy
accuracy = solution_evaluator(group_solution[0], true_solution_test[0])
print(accuracy)

0.6


In [1198]:
adaptation_degree(all_solutions[0], group_solution[0])

0.4

## ABM Model

In [1199]:
def exercise_run(student_list: list, max_ticks: int, n_part_exercises: int, range_solution: list, n_simulations: int):
    '''
    function to run one simulation of a study group completing the exercise (from individual to group solution)
    '''

    #### ---- LISTS ---- #### 
    ## ---- group level ---- ###
    group_accuracy_list = []
    n_tick_list = []
    weak_synergy_list = []
    strong_synergy_list = []
    consensus_list = []

    ## ---- individual level ---- ###
    alfa_accuracy_list = []
    beta_accuracy_list = []
    charlie_accuracy_list = []
    delta_accuracy_list = []

    alfa_adaptation_list = []
    beta_adaptation_list = []
    charlie_adaptation_list = []
    delta_adaptation_list = []

    # ---- starting simulation ---- #
    for i in range(n_simulations):
        ## giving the study group personality values and skills ##
        studygroup = StudyGroup(student_list)
        
        ## ---- SOLUTIONS ---- ##
        ## generating the true solution ## 
        true_solution = true_solution_generator(n_part_exercises, range_solution)

        ## generating individual solutions ##
        all_solutions = individual_solutions_generator(studygroup, true_solution[0], true_solution[1])

        ## evaluating own solutions ##        
        alfa_accuracy = solution_evaluator(all_solutions[0], true_solution[0])
        beta_accuracy = solution_evaluator(all_solutions[1], true_solution[0])
        charlie_accuracy = solution_evaluator(all_solutions[2], true_solution[0])
        delta_accuracy = solution_evaluator(all_solutions[3], true_solution[0])

        all_accuracies = [alfa_accuracy, beta_accuracy, charlie_accuracy, delta_accuracy]
        
        alfa_accuracy_list.append(alfa_accuracy)
        beta_accuracy_list.append(beta_accuracy)
        charlie_accuracy_list.append(charlie_accuracy)
        delta_accuracy_list.append(delta_accuracy)

        ## generating group solution ##
        group_solution = collaborative_solution(studygroup, max_ticks)
        n_tick_list.append(group_solution[1]) # appending the number of ticks
        consensus_list.append(group_solution[2]) # appending whether a consensus was reached or not
        
        ## ---- MEASURES ---- ##
        ## calculating degree of adaptation ## 
        alfa_adaption = adaptation_degree(all_solutions[0], group_solution[0])
        beta_adaption = adaptation_degree(all_solutions[1], group_solution[0])
        charlie_adaption = adaptation_degree(all_solutions[2], group_solution[0])
        delta_adaption = adaptation_degree(all_solutions[3], group_solution[0])

        alfa_adaptation_list.append(alfa_adaption)
        beta_adaptation_list.append(beta_adaption)
        charlie_adaptation_list.append(charlie_adaption)
        delta_adaptation_list.append(delta_adaption)

        ## calculating the accuracy of the group solution ##
        group_accuracy = solution_evaluator(group_solution[0], true_solution[0])
        group_accuracy_list.append(group_accuracy) #appending the group accuracies

        # calculating the gain from the individual solutions to the group solution
        weak_synergy, strong_synergy = synergy_calculator(all_accuracies, group_accuracy)
        weak_synergy_list.append(weak_synergy)
        strong_synergy_list.append(strong_synergy)
        

    
    diversity_score = diversity_score_calculator(student_list)

    ## calculating the mean and standard deivations ##
    # ---- group level ---- #
    avg_group_accuracy = np.mean(group_accuracy_list)
    std_group_accuracy = np.std(group_accuracy_list)
    
    avg_group_n_tick = np.mean(n_tick_list)
    std_group_n_tick = np.std(n_tick_list)

    avg_consensus = np.mean(consensus_list)
    std_consensus = np.std(consensus_list)

    avg_weak_synergy  = np.mean(weak_synergy_list)
    std_weak_synergy = np.std(weak_synergy_list)

    avg_strong_synergy  = np.mean(strong_synergy_list)
    std_strong_synergy = np.std(strong_synergy_list)
    
    # ---- individual level ---- #
    avg_alfa_accuracy = np.mean(alfa_accuracy_list)
    std_alfa_accuracy = np.std(alfa_accuracy_list)

    avg_beta_accuracy = np.mean(beta_accuracy_list)
    std_beta_accuracy = np.std(beta_accuracy_list)

    avg_charlie_accuracy = np.mean(charlie_accuracy_list)
    std_charlie_accuracy = np.std(charlie_accuracy_list)

    avg_delta_accuracy = np.mean(delta_accuracy_list)
    std_delta_accuracy = np.std(delta_accuracy_list)

    avg_alfa_adaptation = np.mean(alfa_adaptation_list)
    avg_beta_adaptation = np.mean(beta_adaptation_list)
    avg_charlie_adaptation = np.mean(charlie_adaptation_list)
    avg_delta_adaptation = np.mean(delta_adaptation_list)
    
    ## dictionaries ## 
    exercise_run_dict_group = {
    'alfa': student_list[0], 
    'beta':student_list[1], 
    'charlie':student_list[2], 
    'delta':student_list[3], 
    'avg_accuracy': avg_group_accuracy, 
    'std_accuracy': std_group_accuracy, 
    'avg_n_tick': avg_group_n_tick, 
    'std_n_tick': std_group_n_tick, 
    'avg_consensus': avg_consensus,
    'std_consensus': std_consensus,
    'avg_weak_synergy': avg_weak_synergy,
    'std_weak_synergy': std_weak_synergy, 
    'avg_strong_synergy': avg_strong_synergy,
    'std_strong_synergy': std_strong_synergy,
    'diversity_score': diversity_score}

    exercise_run_list_ind = list(zip(student_list, 
                                    [student_list, student_list, student_list, student_list], 
                                    [avg_alfa_accuracy, avg_beta_accuracy, avg_charlie_accuracy, avg_delta_accuracy],
                                    [std_alfa_accuracy, std_beta_accuracy, std_charlie_accuracy, std_delta_accuracy], 
                                    [avg_alfa_adaptation, avg_beta_adaptation, avg_charlie_adaptation, avg_delta_adaptation]))
    
    return exercise_run_dict_group, exercise_run_list_ind

In [1200]:
def abm_model(condition, max_ticks: int, n_part_exercises: int, range_solution: list, n_simulations: int):
    condition_group_df = pd.DataFrame(columns = ['alfa', 'beta', 'charlie', 'delta', 'avg_accuracy', 'std_accuracy', 'avg_n_tick', 'std_n_tick', 'avg_consensus', 'std_consensus', 'avg_weak_synergy', 'std_weak_synergy', 'avg_strong_synergy', 'std_strong_synergy', 'diversity_score'])
    condition_ind_df = pd.DataFrame(columns=['type', 'studygroup', 'avg_accuracy', 'std_accuracy', 'avg_adaptation']) #'avg_accuracy', 'contribution', 'n_presentations'])
        
    for i in (range(len(condition.index))):
        student_list = list(condition.iloc[i][1:5])
        
        ## Getting data frames for the group and individual levels ##
        exercise_run_dict_group, exercise_run_list_ind = exercise_run(student_list, max_ticks, n_part_exercises, range_solution, n_simulations) #creating dictionary
        
        exercise_run_group_df = pd.DataFrame(exercise_run_dict_group, index = [i]) #creating dataframe
        condition_group_df = pd.concat([condition_group_df, exercise_run_group_df], ignore_index = True) #concatenating dataframes
        
        exercise_run_ind_df = pd.DataFrame(exercise_run_list_ind, columns=['type', 'studygroup', 'avg_accuracy', 'std_accuracy', 'avg_adaptation']) #'avg_accuracy', 'contribution', 'n_presentations'])
        condition_ind_df = pd.concat([condition_ind_df, exercise_run_ind_df], ignore_index = True)

    return condition_ind_df, condition_group_df


In [1201]:
homogenous_ind_df, homogeous_group_df = abm_model(condition = homogenous, 
        max_ticks = 20, 
        n_part_exercises = 10, 
        range_solution = [1, 9], 
        n_simulations = 100
        )

KeyboardInterrupt: 

In [ ]:
sorted_wsynergy_homogenous = homogeous_group_df.sort_values("avg_accuracy", ascending = True)
sorted_wsynergy_homogenous

homogenous_ind_df.sort_values("avg_adaptation", ascending = True)

,type,studygroup,avg_accuracy,std_accuracy,avg_adaptation
47,INTJ,"[INTJ, INTJ, INTJ, INTJ]",0.897,0.097422,0.178
44,INTJ,"[INTJ, INTJ, INTJ, INTJ]",0.890,0.097468,0.179
38,ENTJ,"[ENTJ, ENTJ, ENTJ, ENTJ]",0.885,0.112583,0.181
46,INTJ,"[INTJ, INTJ, INTJ, INTJ]",0.903,0.101445,0.189
28,INFJ,"[INFJ, INFJ, INFJ, INFJ]",0.888,0.101272,0.190
...,...,...,...,...,...
61,ISTP,"[ISTP, ISTP, ISTP, ISTP]",0.738,0.149519,0.387
58,ISFP,"[ISFP, ISFP, ISFP, ISFP]",0.740,0.133417,0.390
56,ISFP,"[ISFP, ISFP, ISFP, ISFP]",0.748,0.144554,0.394
60,ISTP,"[ISTP, ISTP, ISTP, ISTP]",0.760,0.144914,0.396


In [ ]:
fifty_ind, fifty_group = abm_model(condition = fiftyfifty, 
        max_ticks = 20, 
        n_part_exercises = 10, 
        range_solution = [1, 9], 
        n_simulations = 100
        )

In [ ]:
sorted_wsynergy_fifty = fifty_group.sort_values("avg_accuracy", ascending = True)
sorted_wsynergy_fifty

,type,studygroup,avg_accuracy,std_accuracy,avg_adaptation
61,ENTJ,"[ENTJ, ENTJ, ISTJ, ISTJ]",0.893,0.107009,0.194
45,ENFJ,"[ENFJ, ENFJ, INFP, INFP]",0.886,0.110472,0.198
13,ENFJ,"[ENFJ, ENFJ, ISTP, ISTP]",0.896,0.104805,0.199
12,ENFJ,"[ENFJ, ENFJ, ISTP, ISTP]",0.885,0.108051,0.204
60,ENTJ,"[ENTJ, ENTJ, ISTJ, ISTJ]",0.880,0.095917,0.207
...,...,...,...,...,...
57,ESTP,"[ESTP, ESTP, ISFJ, ISFJ]",0.760,0.135647,0.360
56,ESTP,"[ESTP, ESTP, ISFJ, ISFJ]",0.727,0.166045,0.363
35,ISTP,"[ESFJ, ESFJ, ISTP, ISTP]",0.744,0.145822,0.371
51,ISFP,"[ESTJ, ESTJ, ISFP, ISFP]",0.760,0.125698,0.375


In [ ]:
hetero_ind, hetero_group = abm_model(condition = heterogenous, 
        max_ticks = 20, 
        n_part_exercises = 10, 
        range_solution = [1, 9], 
        n_simulations = 100
        )

In [ ]:
sorted_wsynergy_hetero = hetero_group.sort_values("avg_accuracy", ascending = True)
sorted_wsynergy_hetero

,alfa,beta,charlie,delta,avg_accuracy,std_accuracy,avg_n_tick,std_n_tick,avg_consensus,std_consensus,avg_weak_synergy,std_weak_synergy,avg_strong_synergy,std_strong_synergy,diversity_score
9,ESTJ,ISFP,ISTJ,ESFP,0.768,0.130292,18.14,7.782056,0.86,0.346987,-0.01125,0.116102,-0.143,0.132102,6
5,ESTJ,ESTP,ISTP,INTJ,0.794,0.117320,17.70,7.555792,0.89,0.312890,-0.00025,0.112833,-0.141,0.128915,5
2,ISTJ,ISTP,ESFP,ENTJ,0.795,0.132193,19.45,7.001964,0.89,0.312890,-0.00450,0.121623,-0.139,0.127197,6
13,ENTP,INTP,ESFP,ISFP,0.804,0.121589,17.86,7.477994,0.93,0.255147,0.01750,0.111999,-0.111,0.127980,6
6,ENFP,ISFP,ESFP,INFP,0.810,0.117047,13.96,6.580152,0.95,0.217945,0.01525,0.104337,-0.112,0.124322,4
12,ESFJ,ISFJ,ENFP,INFP,0.810,0.123693,15.63,7.036555,0.98,0.140000,0.00875,0.101512,-0.113,0.108310,6
1,ISFJ,ESTP,ISFP,ENFJ,0.811,0.131069,17.20,8.168231,0.85,0.357071,-0.00075,0.117975,-0.134,0.120183,6
8,ESFJ,ISTP,ESTP,ENTP,0.820,0.126491,18.56,7.373357,0.91,0.286182,0.02675,0.120688,-0.096,0.137055,4
4,ESFJ,ISFJ,ISTJ,INTP,0.826,0.125395,17.79,7.424682,0.90,0.300000,-0.00475,0.129648,-0.121,0.142334,5
15,ENTJ,INTJ,ESTP,ISTP,0.828,0.123353,16.29,7.250234,0.93,0.255147,0.01850,0.117931,-0.120,0.131909,6


In [ ]:
homogenous_mean_acc = np.mean(homogeous_group_df['avg_accuracy'])
fifty_mean_acc = np.mean(fifty_group['avg_accuracy'])
hetero_mean_acc = np.mean(hetero_group['avg_accuracy'])

homogenous_std_acc = np.std(homogeous_group_df['avg_accuracy'])
fifty_std_acc = np.std(fifty_group['avg_accuracy'])
hetero_std_acc = np.std(hetero_group['avg_accuracy'])

homogenous_mean_tick = np.mean(homogeous_group_df['avg_n_tick'])
fifty_mean_tick = np.mean(fifty_group['avg_n_tick'])
hetero_mean_tick = np.mean(hetero_group['avg_n_tick'])

homogenous_std_tick = np.std(homogeous_group_df['avg_n_tick'])
fifty_std_tick = np.std(fifty_group['avg_n_tick'])
hetero_std_tick = np.std(hetero_group['avg_n_tick'])

print(tuple([homogenous_mean_acc, fifty_mean_acc, hetero_mean_acc]))
print(tuple([homogenous_std_acc, fifty_std_acc, hetero_std_acc]))

print(tuple([homogenous_mean_tick, fifty_mean_tick, hetero_mean_tick]))
print(tuple([homogenous_std_tick, fifty_std_tick, hetero_std_tick]))

(0.8166250000000002, 0.8207500000000001, 0.8245625000000002)
(0.046472942396624764, 0.029085004727522457, 0.028122430438175194)
(11.745624999999999, 17.392500000000002, 15.791875)
(2.8001137672200036, 2.301272745677921, 2.325035534002652)
